In [1]:
import geopandas as gpd
import numpy as np

lgas = gpd.read_file("/home/gonzalo/Downloads/map (1).geojson").to_crs("EPSG:4326")
lgas_bounds = lgas.bounds
bounds_all = np.min(lgas_bounds.minx), np.min(lgas_bounds.miny), np.max(lgas_bounds.maxx), np.max(lgas_bounds.maxy)
bounds_all

(152.7978515625, -29.463514026304715, 153.8525390625, -28.478348692223175)

In [2]:
lgas.explore()

In [3]:
import itertools
from ml4floods.data.copernicusEMS.activations import generate_polygon
from shapely.geometry import Polygon

import warnings
warnings.filterwarnings('ignore', 'Geometry is in a ', UserWarning)

step = 0.20, 0.20
size = 0.21

pols_all = []
for idx, (x_left,y_bottom) in enumerate(itertools.product(np.arange(bounds_all[0], bounds_all[2], step[0]),
                                                          np.arange(bounds_all[1],bounds_all[3], step[1]))):
    bounds_iter = (x_left, y_bottom, x_left+size, y_bottom+size)
    pol = Polygon(generate_polygon(bounds_iter))
    
    
    # TODO do not include pol if it does not overlap the lgas
    lgas_overlap_polygons = lgas.intersection(pol)
    lgas_overlap_polygons = lgas_overlap_polygons[~lgas_overlap_polygons.is_empty].copy()
    if lgas_overlap_polygons.shape[0] == 0:
        # print(f"{idx} does not overlap the LGAS")
        continue
    area_overlap_lga = lgas_overlap_polygons.area.sum() / pol.area
    
    if area_overlap_lga<=.3:
        continue
     
    pols_all.append({"geometry": pol, "name":f"AUTOAOI{idx:04d}"})
    
    
pols_all = gpd.GeoDataFrame(pols_all,crs="EPSG:4326")
print(pols_all.shape)
pols_all.to_file("/home/gonzalo/Downloads/autoaois.geojson",driver="GeoJSON")

(25, 2)


In [4]:
pols_all.explore()